# This notebook runs ...

In [ ]:
import os
import sys
import pathlib
import yaml
import subprocess
import re
from collections import defaultdict

import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import umap.umap_ as umap
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from matplotlib_venn import venn2, venn3
import seaborn as sns

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU
import tensorflow as tf
from tensorflow.keras.utils import plot_model

## Metadata Column Definition

In [ ]:
CELL_TYPE_COL = 'encode_celltype'
SAMPLE_ID_COL = 'sample_id'
STIM_COL = 'stim'

GENE_ID_COL = 'gene_ids'

DATASPLIT_COL = 'isTraining'

## Define paths to Required Input Files

### Load config
The config file specifies the path to data and software repo (due to currently in active development)

In [ ]:
# Get the root directory of the analysis repository
REPO_ROOT = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()
REPO_ROOT = pathlib.Path(REPO_ROOT)

CONFIG_FILE = REPO_ROOT / 'config.yml'
assert CONFIG_FILE.exists(), f"Config file not found at {CONFIG_FILE}"

with open(CONFIG_FILE, 'r') as file:
    config_dict = yaml.safe_load(file)

In [ ]:
buddi_fork_path = config_dict['software_path']['buddi_HGSC']
buddi_fork_path = pathlib.Path(buddi_fork_path)
assert buddi_fork_path.exists(), f"buddi fork not found at {buddi_fork_path}"

sys.path.insert(0, str(buddi_fork_path))
# this is quite ugly, once activate modifications are done this will be changed
# to a proper installation + import
from buddi.plotting import validation_plotting as vp
from buddi.models.buddi4 import build_buddi4, fit_buddi4
# from prototype_buddi_dataset import *
from buddi.dataset.dataset import dataset_generator, dataset_generator_unsupervised, get_output_signature, get_output_signature_unsupervised
from buddi.dataset.utils import split_dataset
from buddi.plotting.plot_latent_space import plot_latent_spaces_buddi4

### Check for Pre-Processing Output presence

In [ ]:
PREPROCESSING_OUTPUT_PATH = REPO_ROOT / 'processed_data'
assert PREPROCESSING_OUTPUT_PATH.exists(), f"Preprocessing output path {PREPROCESSING_OUTPUT_PATH} does not exist"

SC_AUGMENTED_DATA_PATH = PREPROCESSING_OUTPUT_PATH / 'sc_augmented'
assert SC_AUGMENTED_DATA_PATH.exists(), f"Single cell augmented data path {SC_AUGMENTED_DATA_PATH} does not exist"
pseudobulk_files = list(SC_AUGMENTED_DATA_PATH.glob('*_splits.pkl'))
assert len(pseudobulk_files) > 0, f"No pseudobulk files found in {SC_AUGMENTED_DATA_PATH}"

BULK_FORMAT_DATA_PATH = PREPROCESSING_OUTPUT_PATH / 'bulk_formatted'
assert BULK_FORMAT_DATA_PATH.exists(), f"Bulk format data path {BULK_FORMAT_DATA_PATH} does not exist"
formatted_bulk_files = list(BULK_FORMAT_DATA_PATH.glob('*.h5ad'))
assert len(formatted_bulk_files) > 0, f"No formatted bulk files found in {BULK_FORMAT_DATA_PATH}"

### Hard-coded path to Cibersortx signature gene file (TODO make this more elegant?)

In [ ]:
CIBERSORTX_SIG_GENE_FILE = pathlib.Path(config_dict['data_path']['sc_data_path']) /\
    'GSE154600_cibersortx_output' /\
    'CIBERSORTx_Job4_GSM4675273_cibersortx_sc_reference_input_inferred_phenoclasses.CIBERSORTx_Job4_GSM4675273_cibersortx_sc_reference_input_inferred_refsample.bm.K999.txt'
assert CIBERSORTX_SIG_GENE_FILE.exists(), f"CIBERSORTx signature gene file not found at {CIBERSORTX_SIG_GENE_FILE}"

## Load Processed Data and Feature select using cibersortx

### Cibersortx signature genes

In [ ]:
cibersortx_sig_df = pd.read_csv(CIBERSORTX_SIG_GENE_FILE, sep='\t', header=0)
cibersortx_sig_genes = cibersortx_sig_df['NAME'].values.tolist()
print(f"{len(cibersortx_sig_genes)} signature genes from CIBERSORTx")

### Processed Bulk

In [ ]:
## TODO support for multiple formatted bulk files
processed_bulk = sc.read(formatted_bulk_files[0]) # Assumes only single file for processed bulk

In [ ]:
bulk_sample_ids = processed_bulk.obs[SAMPLE_ID_COL].unique()
n_bulk_samples = len(bulk_sample_ids)
print(f"Number of bulk samples: {n_bulk_samples}")

bulk_stims = processed_bulk.obs[STIM_COL].unique()
n_bulk_stims = len(bulk_stims)
print(f"Number of bulk stims: {n_bulk_stims}")

bulk_genes = processed_bulk.var[GENE_ID_COL].values.tolist()
print(f"Number of bulk genes: {len(bulk_genes)}")

### Pseudobulks

In [ ]:
sc_gene_file = SC_AUGMENTED_DATA_PATH.glob('*_genes.pkl')
sc_gene_file = list(sc_gene_file)[0]
sc_genes = pd.read_pickle(sc_gene_file)
print(f"Number of single cell genes: {len(sc_genes)}")


### Visualize the overlap of genes

In [ ]:
venn3(
    [
        set(sc_genes),
        set(bulk_genes),
        set(cibersortx_sig_genes)
    ],
    set_labels=['Single Cell', 'Bulk', 'CIBERSORTx']
)

### Subset bulk

In [ ]:
sc_bulk_intersection_genes = set(sc_genes).intersection(set(bulk_genes))
sc_bulk_intersection_genes = list(sc_bulk_intersection_genes)
cibersortx_sig_genes = set(cibersortx_sig_genes).intersection(sc_bulk_intersection_genes)
cibersortx_sig_genes = list(cibersortx_sig_genes)

print(f"Number of genes in intersection of single cell and bulk: {len(sc_bulk_intersection_genes)}")

## Subset the bulk data to only include the intersection genes
processed_bulk = processed_bulk[
    :, 
    processed_bulk.var[GENE_ID_COL].isin(sc_bulk_intersection_genes)]

X_bulk_train = pd.DataFrame(
    processed_bulk.X,
    index=processed_bulk.obs[SAMPLE_ID_COL],
    columns=processed_bulk.var[GENE_ID_COL]
)

# No Y for real bulk

# Add metadata columns 
## TODO: perhaps move this to preprocessing?
meta_bulk_train = processed_bulk.obs.loc[:,[SAMPLE_ID_COL, STIM_COL]]
meta_bulk_train['isTraining'] = 'Train'
meta_bulk_train['cell_prop_type'] = 'bulk'
meta_bulk_train['samp_type'] = 'bulk'

### Load Pseudobulk

In [ ]:
pattern = re.compile(r'^(.*)_((?:Train)|(?:Test))_((?:meta)|(?:prop)|(?:pseudo))_splits\.pkl$')

# Temporary dictionary to group by sample_id
temp_dict = defaultdict(lambda: defaultdict(dict))

for path in pseudobulk_files:
    file = path.name
    match = pattern.match(file)
    if match:
        sample_id, datasplit, datatype = match.groups()
        temp_dict[datasplit][sample_id][datatype] = path

# Organizing the sorted results
grouped_files = {
    "Train": {"meta": [], "prop": [], "pseudo": []},
    "Test": {"meta": [], "prop": [], "pseudo": []}
}

# Sort within each datasplit by sample_id and organize the lists
for datasplit in ["Train", "Test"]:
    sorted_samples = sorted(temp_dict[datasplit].keys())  # Sort by sample_id
    for sample_id in sorted_samples:
        for datatype in ["meta", "prop", "pseudo"]:
            if datatype in temp_dict[datasplit][sample_id]:
                grouped_files[datasplit][datatype].append(temp_dict[datasplit][sample_id][datatype])

meta_sc_train = pd.concat(
    [pd.read_pickle(file) for file in grouped_files['Train']['meta']]
)
meta_sc_train['stim'] = 'white' # ugly fix
Y_sc_train = pd.concat(
    [pd.read_pickle(file) for file in grouped_files['Train']['prop']]
)
X_sc_train = pd.concat(
    [pd.read_pickle(file).loc[:, sc_bulk_intersection_genes] for file in grouped_files['Train']['pseudo']]
)

### Concatenate X, Y and Encode Metadata

In [ ]:
# Generate place holder for bulk proportion (not used)
Y_bulk_dummy = pd.DataFrame(
    np.zeros((X_bulk_train.shape[0], Y_sc_train.shape[1])),
    columns=Y_sc_train.columns
)

X_concat = pd.concat([X_bulk_train, X_sc_train])
Y_concat = pd.concat([Y_bulk_dummy, Y_sc_train])
meta_concat = pd.concat([meta_bulk_train, meta_sc_train])

# save gene and cell type names
X_gene_names = X_concat.columns.to_list()
Y_cell_type_names = Y_concat.columns.to_list()

In [ ]:
## get the top variable genes
X_colmean = X_concat.values.mean(axis=0) # mean across samples
X_colvar = X_concat.values.var(axis=0) # variance across samples
# coefficient of variation which is the var to mean ratio
X_CoV = np.array(np.divide(X_colvar, X_colmean+0.001)) 

# need to get the genes such that
# the union of the highly variable and the
# CIBERSORTx genes are 7000 total
num_genes_found = False

gene_df = pd.DataFrame(X_gene_names, columns=['gene'])

# start with top 7000 genes by CoV
initial_count = 7000
while not num_genes_found:
    # retrieve the top initial_count genes by CoV
    idx_top = np.argpartition(X_CoV, -initial_count)[-initial_count:]
    # get gene names from idx_top
    top_gene_df = gene_df.iloc[idx_top]

    # produce the union of the top genes and the cibersort genes
    CoV_only = np.union1d(top_gene_df, cibersortx_sig_genes)

    # check if the union is 7000
    if len(CoV_only) == 7000:
        num_genes_found = True
    else:
        # if not decrement the top CoV gene to retrieve by 1
        # in the next iteration, there will be two possibilities
        # 1. the union will have one less gene due to the removed gene being only in the CoV genes
        # 2. the union will have the same number of genes as this iteration due to the removed 
        #    gene being also present in CIBERSORTx genes
        # keep running this loop until the union has 7000 genes
        initial_count = initial_count -1

idx_top = np.argpartition(X_CoV, -initial_count)[-initial_count:] # num_genes to get 7000
gene_df = gene_df.iloc[idx_top]
venn2([set(gene_df.values.flatten()), 
       set(cibersortx_sig_genes)], 
       set_labels = ('Top CoV Genes', 'Cibersort Genes'))
plt.show()

In [ ]:
feature_select_genes = np.union1d(gene_df, cibersortx_sig_genes)

# to numpy matrix
X = X_concat.loc[:,feature_select_genes].to_numpy()
print(X.shape)

Y = Y_concat.to_numpy()

In [ ]:
FIELDS_TO_ENCODE = [SAMPLE_ID_COL, STIM_COL, 'samp_type']

encoded = {}

for field in FIELDS_TO_ENCODE:
    encoder = OneHotEncoder(dtype=int)
    encoded[field] = pd.DataFrame(
        encoder.fit_transform(meta_concat.loc[:,[field]]).toarray(),
        columns=encoder.get_feature_names_out([field])
    )

### Final normalization

In [ ]:
## normalize within sample
clip_upper = np.quantile(X, 0.9)
X_full = np.clip(X, 0, clip_upper)
scaler = MinMaxScaler()
scaler.fit(X_full)

# now normalize with the scaler trained on the 
# training data
X_full = np.clip(X_full, 0, clip_upper)
X_full = scaler.transform(X_full)

In [ ]:
idx_train = np.where(meta_concat.isTraining=="Train")[0] 
idx_bulk = np.where(meta_concat.samp_type == "bulk")[0]

# for unknown proportions; 
# this is bulks used in training
idx_bulk_train = np.intersect1d(idx_bulk, idx_train) 

In [ ]:
idx_train = np.where(meta_concat.isTraining=="Train")[0] 
idx_bulk = np.where(meta_concat.samp_type == "bulk")[0]
idx_sc = np.where(meta_concat.samp_type != "bulk")[0]

# for unknown proportions; 
# this is bulks used in training
idx_bulk_train = np.intersect1d(idx_bulk, idx_train)
np.random.shuffle(idx_bulk_train) # shuffle the indices randomly

X_unkp = X_full[idx_bulk_train,]
label_unkp = encoded['sample_id'].values[idx_bulk_train,]
drug_unkp = encoded['stim'].values[idx_bulk_train,]
bulk_unkp = encoded['samp_type'].values[idx_bulk_train,]
y_unkp = Y[idx_bulk_train,]

# for known proportions
idx_sc_train = np.intersect1d(idx_sc, idx_train)
np.random.shuffle(idx_sc_train) # shuffle the indices randomly

X_kp = X_full[idx_sc_train,]
label_kp = encoded['sample_id'].values[idx_sc_train,]
drug_kp = encoded['stim'].values[idx_sc_train,]
bulk_kp = encoded['samp_type'].values[idx_sc_train,]
y_kp = Y[idx_sc_train,]

# Plot all data

In the plot below, we plot the pseudobulk and real bulk data. Each plot will have a different coloration in order to highlight the different confounders present in the data.
In the first plot, it is colored by sample ID, but since there are >500 real bulk samples, we hide the legend.
The final plot is colored by the cell-type that has the highest proportion in the pseudobulk. 
For the bulk data, this is a random cell-type.

In [ ]:
# ## plot samples
# plot_df = vp.get_tsne_for_plotting(X_full)

# fig, axs = plt.subplots(2, 3, figsize=(30,15))

# Y_temp = np.copy(Y)
# Y_temp = np.argmax(Y_temp, axis=1) 

# vp.plot_tsne(plot_df, meta_concat.sample_id.to_numpy(), axs[0,0], title=f"")
# vp.plot_tsne(plot_df, meta_concat.stim.to_numpy(), axs[0,1], title=f"")
# vp.plot_tsne(plot_df, meta_concat.isTraining.to_numpy(), axs[0,2], title=f"")
# vp.plot_tsne(plot_df, meta_concat.cell_prop_type.to_numpy(), axs[1,0], title=f"")
# vp.plot_tsne(plot_df, meta_concat.samp_type.to_numpy(), axs[1,1], title=f"")
# vp.plot_tsne(plot_df, Y_temp, axs[1,2], title=f"")


# fig.suptitle("All Data, normalized using training data", fontsize=14)

# axs[0,0].legend([],[], frameon=False)

# fig.show()

## Prototyping Training

In [ ]:
n_x = X_full.shape[1]
n_y = Y.shape[1]
n_labels = encoded['sample_id'].shape[1]
n_stims = encoded['stim'].shape[1]
n_samp_types = encoded['samp_type'].shape[1]

In [ ]:
supervised_buddi, unsupervised_buddi = build_buddi4(
    n_x=n_x,
    n_y=n_y,
    n_labels=n_labels,
    n_stims=n_stims,
    n_samp_types=n_samp_types,
    reconstr_loss_fn = tf.keras.losses.mean_squared_error, # default reconstruction loss is mean squared error
    classifier_loss_fn = tf.keras.losses.mean_absolute_error, # default classifier loss is mean absolute error
    label_classifier_loss_fn = tf.keras.losses.binary_crossentropy # only sample id classifier loss is binary crossentropy
)

In [ ]:
plot_model(supervised_buddi, to_file='supervised_buddi.png', show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(unsupervised_buddi, to_file='unsupervised_buddi.png', show_shapes=True, show_layer_names=True)

In [ ]:
dataset_supervised = tf.data.Dataset.from_generator(
    lambda: dataset_generator(X_kp, y_kp, label_kp, drug_kp, bulk_kp),
    output_signature=get_output_signature(X_kp, y_kp, label_kp, drug_kp, bulk_kp)
)

dataset_unsupervised = tf.data.Dataset.from_generator(
    lambda: dataset_generator_unsupervised(X_unkp, label_unkp, drug_unkp, bulk_unkp),
    output_signature=get_output_signature_unsupervised(X_unkp, label_unkp, drug_unkp, bulk_unkp)
)

In [ ]:
# get the training pseudobulks
idx_tmp_bulk = np.where(np.logical_and(meta_concat.isTraining == "Train", meta_concat.samp_type == "sc_ref"))[0]
idx_tmp_bulk = np.random.choice(idx_tmp_bulk, 500, replace=True)

# get the real bulks
idx_tmp = np.where(np.logical_and(meta_concat.isTraining == "Train", meta_concat.samp_type == "bulk"))[0]
idx_tmp = np.random.choice(idx_tmp, 500, replace=True)

# concatenate so we have them all
idx_tmp = np.concatenate((idx_tmp, idx_tmp_bulk))

X_tmp = X_full[idx_tmp,]
meta_tmp = meta_concat.iloc[idx_tmp,]

cell_types = list(Y_concat.columns)
Y_tmp = Y[idx_tmp,]
idx_sc_prop = np.where(meta_tmp.cell_prop_type == "single_celltype")[0]
cell_type_labels = [cell_types[i] for i in np.argmax(Y_tmp[idx_sc_prop, :], axis=1)]
cell_type_col = meta_tmp['cell_prop_type'].values.copy()
cell_type_col[idx_sc_prop] = cell_type_labels
meta_tmp.loc[:,'cell_type'] = cell_type_col

In [ ]:
loss_df_concat = pd.DataFrame()

for _i in range(4):

    all_loss_df = fit_buddi4(
        supervised_buddi, unsupervised_buddi, 
        dataset_supervised, dataset_unsupervised, 
        epochs=50, batch_size=16, shuffle_cache_size=10000
    )
    loss_df_concat = pd.concat([loss_df_concat, all_loss_df], ignore_index=True)

    fig, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=True)

    loss_columns = [
        'X_reconstruction_loss', 
        'classifier_label_loss', 
        'classifier_stim_loss', 
        'classifier_samp_type_loss', 
        'prop_estimator_loss'
    ]

    for ax, col in zip(axes, loss_columns):
        sns.lineplot(
            data=all_loss_df, 
            x='epoch', 
            y=col, 
            hue='type', 
            ax=ax
        )
        ax.set_title(col)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')

    plt.tight_layout()
    plt.show()

    plot_latent_spaces_buddi4(
        unsupervised_buddi,
        X_tmp,
        meta_tmp,
        type='UMAP',
        alpha=1,    
        panel_width=3
    )

    plot_latent_spaces_buddi4(
        unsupervised_buddi,
        X_tmp,
        meta_tmp,
        type='PCA',
        alpha=1,    
        panel_width=3
    )